# Word Embeddings Training: Word2Vec, FastText and GloVe
## NLP at AIMS South Africa 2025
### Based on Lecture 04: Word Representation

This notebook demonstrates:
1. Word2Vec (CBOW and Skip-gram) training
2. FastText training
3. GloVe training
4. Both dummy data and real datasets from HuggingFace

## Installation and Imports

In [ ]:
# Install required packages (uncomment if needed)
# !pip install gensim datasets numpy matplotlib scikit-learn

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
from datasets import load_dataset
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict, Counter
import warnings
warnings.filterwarnings('ignore')

print("All libraries imported successfully!")

---
Getting Training Data
---

---
## 1. Dummy dataset
---

In [ ]:
# Create a simple corpus
dummy_corpus = [
    "the cat sat on the mat",
    "the dog sat on the log",
    "cats and dogs are animals",
    "the cat and the dog are friends",
    "a cat is a pet",
    "a dog is a pet",
    "cats like to sit on mats",
    "dogs like to play with balls",
    "the animal sat on the ground",
    "pets are good friends"
]

# Tokenize sentences
dummy_sentences = [sentence.split() for sentence in dummy_corpus]
print(f"Number of sentences: {len(dummy_sentences)}")
print(f"Example sentence: {dummy_sentences[0]}")

# Build vocabulary
vocab = set(word for sentence in dummy_sentences for word in sentence)
print(f"\nVocabulary size: {len(vocab)}")
print(f"Vocabulary: {sorted(vocab)}")

---
## 2. Real dataset (HuggingFace)
---

We'll use the AG News dataset - a collection of news articles.

In [ ]:
# Load a moderately sized dataset - using AG News (news classification)
print("Loading AG News dataset from HuggingFace...")
dataset = load_dataset("ag_news", split="train[:5000]")  # Using 5000 samples

# Prepare sentences
real_sentences = []
for example in dataset:
    # Tokenize simply by splitting on whitespace and converting to lowercase
    words = example['text'].lower().split()
    if len(words) > 3:  # Filter very short sentences
        real_sentences.append(words)

print(f"Number of sentences: {len(real_sentences)}")
print(f"Example sentence: {' '.join(real_sentences[0][:15])}...")

# Build vocabulary stats
all_words = [word for sentence in real_sentences for word in sentence]
vocab_real = set(all_words)
word_freq = Counter(all_words)

print(f"\nVocabulary size: {len(vocab_real)}")
print(f"Total words: {len(all_words)}")
print(f"\nMost common words:")
for word, count in word_freq.most_common(10):
    print(f"  {word}: {count}")

---
# PART 0: Baseline - TF-IDF Representation
---

Before diving into word embeddings, let's understand traditional TF-IDF representation.

**TF-IDF (Term Frequency-Inverse Document Frequency):**
- **TF**: How often a word appears in a document
- **IDF**: How rare/common a word is across all documents
- **Result**: Sparse vectors that weight important words

**Limitations:**
- High dimensionality (vocabulary size)
- Sparse representations (mostly zeros)
- No semantic meaning ("cat" and "kitten" are completely different)
- Cannot handle out-of-vocabulary words

**Why Word Embeddings?**
Word embeddings solve these problems by learning dense, semantic representations!

## 0.1: TF-IDF on Dummy Data

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Create TF-IDF vectors from dummy corpus
tfidf_vectorizer = TfidfVectorizer(
    max_features=50,  # Limit features for visualization
    lowercase=True,
    stop_words=None   # Keep all words for comparison
)

# Fit and transform
tfidf_matrix = tfidf_vectorizer.fit_transform(dummy_corpus)
feature_names = tfidf_vectorizer.get_feature_names_out()

print("TF-IDF Vectorizer trained!")
print(f"Vocabulary size: {len(feature_names)}")
print(f"Matrix shape: {tfidf_matrix.shape}")
print(f"Matrix sparsity: {(1 - tfidf_matrix.nnz / (tfidf_matrix.shape[0] * tfidf_matrix.shape[1])) * 100:.2f}%")
print(f"\nVocabulary: {list(feature_names)}")

In [ ]:
# Analyze TF-IDF weights for a specific document
doc_idx = 0  # "the cat sat on the mat"
doc_vector = tfidf_matrix[doc_idx].toarray()[0]

print(f"Document: '{dummy_corpus[doc_idx]}'\n")
print("TF-IDF weights:")
word_weights = [(feature_names[i], doc_vector[i]) for i in range(len(feature_names)) if doc_vector[i] > 0]
word_weights.sort(key=lambda x: x[1], reverse=True)

for word, weight in word_weights:
    print(f"  {word}: {weight:.4f}")

In [ ]:
# Find similar documents using TF-IDF
query_doc = "the cat sat on the mat"
query_vector = tfidf_vectorizer.transform([query_doc])

# Compute similarities
similarities = cosine_similarity(query_vector, tfidf_matrix)[0]

print(f"Query: '{query_doc}'\n")
print("Most similar documents:")
for idx in np.argsort(similarities)[::-1][:5]:
    print(f"  [{idx}] {dummy_corpus[idx]}: {similarities[idx]:.4f}")

### TF-IDF Limitations Demonstration

In [ ]:
# Demonstrate lack of semantic understanding
print("TF-IDF LIMITATION: No Semantic Understanding\n")
print("="*60)

# Create word vectors by averaging document vectors where word appears
def get_word_representation_tfidf(word, vectorizer, matrix, corpus):
    """Get a pseudo-vector for a word by averaging TF-IDF vectors of documents containing it"""
    word_docs = [i for i, doc in enumerate(corpus) if word in doc]
    if not word_docs:
        return None
    return matrix[word_docs].mean(axis=0).A1

# Get representations
cat_tfidf = get_word_representation_tfidf('cat', tfidf_vectorizer, tfidf_matrix, dummy_corpus)
dog_tfidf = get_word_representation_tfidf('dog', tfidf_vectorizer, tfidf_matrix, dummy_corpus)
mat_tfidf = get_word_representation_tfidf('mat', tfidf_vectorizer, tfidf_matrix, dummy_corpus)

if cat_tfidf is not None and dog_tfidf is not None and mat_tfidf is not None:
    # Compute similarities
    cat_dog_sim = cosine_similarity([cat_tfidf], [dog_tfidf])[0][0]
    cat_mat_sim = cosine_similarity([cat_tfidf], [mat_tfidf])[0][0]
    
    print(f"Similarity (cat, dog): {cat_dog_sim:.4f}")
    print(f"Similarity (cat, mat): {cat_mat_sim:.4f}")
    print("\n⚠️  Problem: TF-IDF doesn't capture that 'cat' and 'dog' are semantically related!")
    print("    They appear in similar contexts but TF-IDF only looks at word co-occurrence.")
else:
    print("Some words not found in corpus")

print("\n" + "="*60)
print("\n✅ Word embeddings solve this by learning semantic relationships!")

In [ ]:
# Visualize TF-IDF sparsity
plt.figure(figsize=(12, 6))

# Plot sparse matrix
plt.subplot(1, 2, 1)
plt.spy(tfidf_matrix, markersize=5, aspect='auto')
plt.title('TF-IDF Matrix Sparsity\n(Blue = Non-zero values)')
plt.xlabel('Features (Words)')
plt.ylabel('Documents')

# Plot feature distribution
plt.subplot(1, 2, 2)
feature_counts = np.array((tfidf_matrix > 0).sum(axis=0)).flatten()
plt.bar(range(len(feature_counts)), feature_counts)
plt.title('Word Frequency Across Documents')
plt.xlabel('Word Index')
plt.ylabel('Number of Documents')
plt.xticks(rotation=90)

plt.tight_layout()
plt.show()

print(f"\nSparsity: {(1 - tfidf_matrix.nnz / (tfidf_matrix.shape[0] * tfidf_matrix.shape[1])) * 100:.2f}%")
print("Most TF-IDF values are zero (sparse representation)")

## 0.2: TF-IDF on Real Dataset

In [ ]:
# Prepare corpus for TF-IDF
real_corpus_text = [' '.join(sentence) for sentence in real_sentences]

# Create TF-IDF vectors
real_tfidf_vectorizer = TfidfVectorizer(
    max_features=1000,  # Limit to top 1000 features
    max_df=0.8,         # Ignore words that appear in >80% of documents
    min_df=5,           # Ignore words that appear in <5 documents
    lowercase=True
)

real_tfidf_matrix = real_tfidf_vectorizer.fit_transform(real_corpus_text)
real_feature_names = real_tfidf_vectorizer.get_feature_names_out()

print("TF-IDF on Real Data:")
print(f"Vocabulary size: {len(real_feature_names)}")
print(f"Matrix shape: {real_tfidf_matrix.shape}")
print(f"Matrix sparsity: {(1 - real_tfidf_matrix.nnz / (real_tfidf_matrix.shape[0] * real_tfidf_matrix.shape[1])) * 100:.2f}%")
print(f"\nTop 20 features by average TF-IDF:")

# Calculate average TF-IDF per feature
mean_tfidf = np.array(real_tfidf_matrix.mean(axis=0)).flatten()
top_indices = np.argsort(mean_tfidf)[::-1][:20]

for idx in top_indices:
    print(f"  {real_feature_names[idx]}: {mean_tfidf[idx]:.4f}")

In [ ]:
# Document similarity search with TF-IDF
query = "business market economy"
query_vector = real_tfidf_vectorizer.transform([query])

# Find most similar documents
similarities = cosine_similarity(query_vector, real_tfidf_matrix)[0]
top_doc_indices = np.argsort(similarities)[::-1][:5]

print(f"Query: '{query}'\n")
print("Most similar documents (TF-IDF):")
for idx in top_doc_indices:
    doc_preview = ' '.join(real_sentences[idx][:15]) + '...'
    print(f"\n[{idx}] Similarity: {similarities[idx]:.4f}")
    print(f"  {doc_preview}")

## 0.3: TF-IDF vs Word Embeddings Comparison

In [ ]:
# Side-by-side comparison
print("="*80)
print("COMPARISON: TF-IDF vs Word Embeddings (Word2Vec, FastText, GloVe)")
print("="*80)

comparison_data = [
    ["Characteristic", "TF-IDF", "Word Embeddings"],
    ["-" * 20, "-" * 30, "-" * 30],
    ["Representation", "Sparse (mostly zeros)", "Dense (all values meaningful)"],
    ["Dimensionality", "High (vocab size)", "Low (50-300 typically)"],
    ["Semantic Meaning", "❌ No", "✅ Yes"],
    ["Word Similarity", "❌ Cannot compare words", "✅ Can measure similarity"],
    ["OOV Handling", "❌ No (except FastText)", "✅ FastText handles OOV"],
    ["Training Required", "❌ No (count-based)", "✅ Yes (learned)"],
    ["Computation", "Fast (simple counting)", "Slower (neural networks)"],
    ["Memory Usage", "High (sparse but large)", "Low (dense but small)"],
    ["Best For", "Document retrieval, search", "Semantic tasks, similarity"],
    ["Context", "Document-level", "Word-level context"],
]

for row in comparison_data:
    print(f"{row[0]:<20} | {row[1]:<30} | {row[2]:<30}")

print("\n" + "="*80)
print("\n💡 Key Insight:")
print("   TF-IDF: Good for finding documents with specific keywords")
print("   Embeddings: Good for understanding meaning and semantic relationships")
print("\n   In practice, both can be used together!")
print("="*80)

In [ ]:
# Visualize dimensionality difference
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# TF-IDF vector for one document
sample_tfidf = real_tfidf_matrix[0].toarray()[0]
axes[0].bar(range(len(sample_tfidf)), sample_tfidf, width=1.0, alpha=0.7)
axes[0].set_title(f'TF-IDF Vector (Sparse)\nDimensions: {len(sample_tfidf)}, Non-zeros: {np.count_nonzero(sample_tfidf)}')
axes[0].set_xlabel('Feature Index')
axes[0].set_ylabel('TF-IDF Weight')
axes[0].set_xlim(0, min(200, len(sample_tfidf)))  # Show first 200

# Word embedding vector (will be added after training)
# Placeholder - will show actual embedding after Word2Vec training
axes[1].text(0.5, 0.5, 'Word Embeddings\n(Dense vectors)\nWill be shown after training!\n\nTypically 50-300 dimensions\nAll values are meaningful', 
             ha='center', va='center', fontsize=12, transform=axes[1].transAxes)
axes[1].set_title('Word Embedding Vector (Dense)')
axes[1].axis('off')

plt.tight_layout()
plt.show()

print(f"\nTF-IDF: {len(sample_tfidf)} dimensions, {(1 - np.count_nonzero(sample_tfidf)/len(sample_tfidf))*100:.1f}% sparse")
print(f"Word Embeddings: 50-300 dimensions, 0% sparse (all values meaningful)")

---
# PART 1: Training on Dummy Data
---

## 1.1: Word2Vec CBOW on Dummy Data

CBOW (Continuous Bag of Words) predicts the target word from context words.

In [ ]:
# Train CBOW model
# sg=0 means CBOW (Continuous Bag of Words)
cbow_model = Word2Vec(
    sentences=dummy_sentences,
    vector_size=50,          # Embedding dimension
    window=2,                # Context window size
    min_count=1,             # Minimum word frequency
    sg=0,                    # 0 = CBOW, 1 = Skip-gram
    epochs=100,              # Number of training iterations
    seed=42
)

print(f"CBOW Model trained!")
print(f"Vector size: {cbow_model.vector_size}")
print(f"Vocabulary size: {len(cbow_model.wv)}")

# Get word vector
cat_vector = cbow_model.wv['cat']
print(f"\nVector for 'cat' (first 10 dims): {cat_vector[:10]}")

# Find similar words
print("\nMost similar words to 'cat':")
for word, score in cbow_model.wv.most_similar('cat', topn=5):
    print(f"  {word}: {score:.4f}")

## 1.2: Word2Vec Skip-gram on Dummy Data

Skip-gram predicts context words from the target word.

In [ ]:
# Train Skip-gram model
# sg=1 means Skip-gram
skipgram_model = Word2Vec(
    sentences=dummy_sentences,
    vector_size=50,
    window=2,
    min_count=1,
    sg=1,                    # 1 = Skip-gram
    epochs=100,
    seed=42
)

print(f"Skip-gram Model trained!")

# Find similar words
print("\nMost similar words to 'cat' (Skip-gram):")
for word, score in skipgram_model.wv.most_similar('cat', topn=5):
    print(f"  {word}: {score:.4f}")

## 1.3: GloVe Implementation on Dummy Data

GloVe learns embeddings from global co-occurrence statistics.

In [ ]:
class SimpleGloVe:
    """Simplified GloVe implementation for educational purposes"""
    
    def __init__(self, vector_size=50, window_size=2, learning_rate=0.05, epochs=100):
        self.vector_size = vector_size
        self.window_size = window_size
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.word_to_id = {}
        self.id_to_word = {}
        
    def build_vocab(self, sentences):
        """Build vocabulary from sentences"""
        words = set(word for sentence in sentences for word in sentence)
        self.word_to_id = {word: idx for idx, word in enumerate(sorted(words))}
        self.id_to_word = {idx: word for word, idx in self.word_to_id.items()}
        self.vocab_size = len(self.word_to_id)
        
    def build_cooccurrence_matrix(self, sentences):
        """Build co-occurrence matrix"""
        cooccur = defaultdict(float)
        
        for sentence in sentences:
            for i, word in enumerate(sentence):
                if word not in self.word_to_id:
                    continue
                word_id = self.word_to_id[word]
                
                # Look at context words within window
                start = max(0, i - self.window_size)
                end = min(len(sentence), i + self.window_size + 1)
                
                for j in range(start, end):
                    if i != j and sentence[j] in self.word_to_id:
                        context_id = self.word_to_id[sentence[j]]
                        distance = abs(i - j)
                        # Weight by distance
                        cooccur[(word_id, context_id)] += 1.0 / distance
        
        return cooccur
    
    def train(self, sentences):
        """Train GloVe embeddings"""
        self.build_vocab(sentences)
        cooccur = self.build_cooccurrence_matrix(sentences)
        
        # Initialize word vectors and biases randomly
        np.random.seed(42)
        self.W = np.random.randn(self.vocab_size, self.vector_size) * 0.01
        self.W_context = np.random.randn(self.vocab_size, self.vector_size) * 0.01
        self.b = np.zeros(self.vocab_size)
        self.b_context = np.zeros(self.vocab_size)
        
        # Training loop
        print(f"Training GloVe on {len(cooccur)} co-occurrence pairs...")
        
        for epoch in range(self.epochs):
            total_loss = 0
            
            for (i, j), X_ij in cooccur.items():
                # Compute weighted loss
                weight = min((X_ij / 100.0) ** 0.75, 1.0)  # x_max = 100, alpha = 0.75
                
                # Compute dot product
                diff = (self.W[i] @ self.W_context[j] + 
                       self.b[i] + self.b_context[j] - np.log(X_ij))
                
                loss = weight * (diff ** 2)
                total_loss += loss
                
                # Gradients
                grad = weight * diff
                
                # Update parameters
                self.W[i] -= self.learning_rate * grad * self.W_context[j]
                self.W_context[j] -= self.learning_rate * grad * self.W[i]
                self.b[i] -= self.learning_rate * grad
                self.b_context[j] -= self.learning_rate * grad
            
            if (epoch + 1) % 20 == 0:
                print(f"  Epoch {epoch+1}/{self.epochs}, Loss: {total_loss:.4f}")
        
        # Final embeddings: sum of word and context vectors
        self.embeddings = self.W + self.W_context
        
    def get_vector(self, word):
        """Get embedding for a word"""
        if word in self.word_to_id:
            return self.embeddings[self.word_to_id[word]]
        return None
    
    def most_similar(self, word, topn=5):
        """Find most similar words"""
        if word not in self.word_to_id:
            return []
        
        word_vec = self.get_vector(word).reshape(1, -1)
        similarities = cosine_similarity(word_vec, self.embeddings)[0]
        
        # Get top similar words (excluding the word itself)
        word_id = self.word_to_id[word]
        similarities[word_id] = -1  # Exclude self
        
        top_indices = np.argsort(similarities)[-topn:][::-1]
        return [(self.id_to_word[idx], similarities[idx]) for idx in top_indices]

In [ ]:
# Train GloVe model
glove_model = SimpleGloVe(vector_size=50, window_size=2, epochs=100)
glove_model.train(dummy_sentences)

print("\nMost similar words to 'cat' (GloVe):")
for word, score in glove_model.most_similar('cat', topn=5):
    print(f"  {word}: {score:.4f}")

## 1.4: FastText on Dummy Data

FastText extends Word2Vec by representing words as bags of character n-grams.
This helps with:
- Handling out-of-vocabulary (OOV) words
- Capturing morphological information
- Better representations for rare words

In [ ]:
from gensim.models import FastText

# Train FastText model (CBOW)
fasttext_cbow = FastText(
    sentences=dummy_sentences,
    vector_size=50,
    window=2,
    min_count=1,
    sg=0,                    # 0 = CBOW, 1 = Skip-gram
    min_n=2,                 # Minimum character n-gram length
    max_n=5,                 # Maximum character n-gram length
    epochs=100,
    seed=42
)

print(f"FastText CBOW Model trained!")
print(f"Vector size: {fasttext_cbow.vector_size}")
print(f"Vocabulary size: {len(fasttext_cbow.wv)}")

# Find similar words
print("\nMost similar words to 'cat' (FastText CBOW):")
for word, score in fasttext_cbow.wv.most_similar('cat', topn=5):
    print(f"  {word}: {score:.4f}")

# Test OOV word handling - FastText can generate vectors for unseen words!
print("\nTesting OOV handling:")
oov_word = 'catty'  # Not in training corpus
if oov_word not in fasttext_cbow.wv:
    print(f"'{oov_word}' is not in vocabulary, but FastText can still generate a vector!")
    oov_vector = fasttext_cbow.wv[oov_word]
    print(f"Vector for '{oov_word}' (first 10 dims): {oov_vector[:10]}")
    
    # Find similar words to OOV word
    print(f"\nMost similar words to '{oov_word}':")
    for word, score in fasttext_cbow.wv.most_similar(oov_word, topn=5):
        print(f"  {word}: {score:.4f}")

In [ ]:
# Train FastText Skip-gram model
fasttext_skipgram = FastText(
    sentences=dummy_sentences,
    vector_size=50,
    window=2,
    min_count=1,
    sg=1,                    # Skip-gram
    min_n=2,
    max_n=5,
    epochs=100,
    seed=42
)

print(f"FastText Skip-gram Model trained!")

# Find similar words
print("\nMost similar words to 'cat' (FastText Skip-gram):")
for word, score in fasttext_skipgram.wv.most_similar('cat', topn=5):
    print(f"  {word}: {score:.4f}")

---
# PART 2: Training on Real Dataset
---

## 2.1: Word2Vec CBOW on Real Dataset

In [ ]:
real_cbow = Word2Vec(
    sentences=real_sentences,
    vector_size=100,
    window=5,
    min_count=5,           # Ignore rare words
    sg=0,                  # CBOW
    workers=4,
    epochs=10,
    seed=42
)

print(f"CBOW Model trained on real data!")
print(f"Vocabulary size: {len(real_cbow.wv)}")

# Test word analogies and similarities
test_words = ['business', 'company', 'market', 'government', 'sports']
available_test_words = [w for w in test_words if w in real_cbow.wv]

if available_test_words:
    test_word = available_test_words[0]
    print(f"\nMost similar words to '{test_word}':")
    for word, score in real_cbow.wv.most_similar(test_word, topn=5):
        print(f"  {word}: {score:.4f}")

## 2.2: Word2Vec Skip-gram on Real Dataset

In [ ]:
real_skipgram = Word2Vec(
    sentences=real_sentences,
    vector_size=100,
    window=5,
    min_count=5,
    sg=1,                  # Skip-gram
    workers=4,
    epochs=10,
    seed=42
)

print(f"Skip-gram Model trained on real data!")
print(f"Vocabulary size: {len(real_skipgram.wv)}")

if available_test_words:
    test_word = available_test_words[0]
    print(f"\nMost similar words to '{test_word}' (Skip-gram):")
    for word, score in real_skipgram.wv.most_similar(test_word, topn=5):
        print(f"  {word}: {score:.4f}")

## 2.3: GloVe on Real Dataset

In [ ]:
# Train GloVe on real data
print("Training GloVe on real dataset...")
real_glove = SimpleGloVe(
    vector_size=100,
    window_size=5,
    learning_rate=0.05,
    epochs=50  # Fewer epochs for larger dataset
)

# Note: Training GloVe on large corpus takes time
# For demonstration, we'll use a subset
glove_subset = real_sentences[:1000]  # Use first 1000 sentences
print(f"Training on {len(glove_subset)} sentences...")
real_glove.train(glove_subset)

print("\nGloVe Model trained on real data!")
print(f"Vocabulary size: {real_glove.vocab_size}")

# Test the model
test_words_glove = ['business', 'company', 'market', 'government', 'sports', 'the', 'is', 'to']
available_glove_words = [w for w in test_words_glove if w in real_glove.word_to_id]

if available_glove_words:
    test_word = available_glove_words[0]
    print(f"\nMost similar words to '{test_word}' (GloVe):")
    for word, score in real_glove.most_similar(test_word, topn=5):
        print(f"  {word}: {score:.4f}")

## 2.4: FastText on Real Dataset

In [ ]:
# Train FastText CBOW on real data
real_fasttext_cbow = FastText(
    sentences=real_sentences,
    vector_size=100,
    window=5,
    min_count=5,
    sg=0,                    # CBOW
    min_n=3,                 # Character n-grams from 3 to 6
    max_n=6,
    workers=4,
    epochs=10,
    seed=42
)

print(f"FastText CBOW Model trained on real data!")
print(f"Vocabulary size: {len(real_fasttext_cbow.wv)}")

if available_test_words:
    test_word = available_test_words[0]
    print(f"\nMost similar words to '{test_word}' (FastText):")
    for word, score in real_fasttext_cbow.wv.most_similar(test_word, topn=5):
        print(f"  {word}: {score:.4f}")

In [ ]:
# Train FastText Skip-gram on real data
real_fasttext_skipgram = FastText(
    sentences=real_sentences,
    vector_size=100,
    window=5,
    min_count=5,
    sg=1,                    # Skip-gram
    min_n=3,
    max_n=6,
    workers=4,
    epochs=10,
    seed=42
)

print(f"FastText Skip-gram Model trained on real data!")
print(f"Vocabulary size: {len(real_fasttext_skipgram.wv)}")

if available_test_words:
    test_word = available_test_words[0]
    print(f"\nMost similar words to '{test_word}' (FastText Skip-gram):")
    for word, score in real_fasttext_skipgram.wv.most_similar(test_word, topn=5):
        print(f"  {word}: {score:.4f}")

### Testing FastText's OOV Handling on Real Data

In [ ]:
# Demonstrate FastText's ability to handle OOV words
print("Testing Out-of-Vocabulary (OOV) word handling:\n")

# Create some OOV words based on known words
if 'business' in real_fasttext_cbow.wv:
    oov_tests = ['businesses', 'businessman', 'businesslike', 'unbusiness']
    known_word = 'business'
elif 'company' in real_fasttext_cbow.wv:
    oov_tests = ['companies', 'accompanying', 'companionship']
    known_word = 'company'
else:
    oov_tests = ['testing', 'tested', 'tester']
    known_word = 'test'

print(f"Base word: '{known_word}'\n")

for oov_word in oov_tests:
    if oov_word not in real_fasttext_cbow.wv:
        print(f"OOV Word: '{oov_word}'")
        # FastText can still generate vectors for these!
        similar = real_fasttext_cbow.wv.most_similar(oov_word, topn=3)
        print(f"  Most similar words:")
        for word, score in similar:
            print(f"    {word}: {score:.4f}")
        print()

# Compare with Word2Vec (which cannot handle OOV)
print("\nComparison: Word2Vec cannot handle OOV words")
oov_word = oov_tests[0]
try:
    if oov_word not in real_cbow.wv:
        _ = real_cbow.wv[oov_word]
except KeyError:
    print(f"Word2Vec: Cannot generate vector for '{oov_word}' (KeyError)")

print(f"FastText: Can generate vector for '{oov_word}' using character n-grams!")

### TF-IDF vs Embeddings: Practical Comparison

In [ ]:
# Compare document similarity using TF-IDF vs Word Embeddings
def document_vector_from_embeddings(doc_text, model):
    """Create document vector by averaging word embeddings"""
    words = doc_text.lower().split()
    vectors = []
    for word in words:
        if hasattr(model, 'wv') and word in model.wv:
            vectors.append(model.wv[word])
    if vectors:
        return np.mean(vectors, axis=0)
    return None

# Test query
test_query = "financial market business"
print(f"Query: '{test_query}'\n")
print("="*80)

# TF-IDF search
print("\nTF-IDF Results (keyword matching):")
query_tfidf = real_tfidf_vectorizer.transform([test_query])
tfidf_sims = cosine_similarity(query_tfidf, real_tfidf_matrix)[0]
top_tfidf = np.argsort(tfidf_sims)[::-1][:3]

for idx in top_tfidf:
    preview = ' '.join(real_sentences[idx][:12]) + '...'
    print(f"  [{idx}] Score: {tfidf_sims[idx]:.4f}")
    print(f"       {preview}\n")

# Word2Vec search
print("\nWord2Vec Results (semantic matching):")
query_w2v = document_vector_from_embeddings(test_query, real_cbow)
if query_w2v is not None:
    w2v_sims = []
    for sent in real_sentences:
        sent_vec = document_vector_from_embeddings(' '.join(sent), real_cbow)
        if sent_vec is not None:
            sim = cosine_similarity([query_w2v], [sent_vec])[0][0]
            w2v_sims.append(sim)
        else:
            w2v_sims.append(-1)
    
    top_w2v = np.argsort(w2v_sims)[::-1][:3]
    for idx in top_w2v:
        preview = ' '.join(real_sentences[idx][:12]) + '...'
        print(f"  [{idx}] Score: {w2v_sims[idx]:.4f}")
        print(f"       {preview}\n")

print("="*80)
print("\n💡 Observation:")
print("   - TF-IDF: Finds documents with exact keyword matches")
print("   - Word2Vec: Finds documents with semantic similarity (synonyms, related concepts)")
print("="*80)

In [ ]:
# Quantitative comparison: Storage requirements
import sys

print("\nSTORAGE COMPARISON")
print("="*60)

# TF-IDF storage
tfidf_size = real_tfidf_matrix.data.nbytes + real_tfidf_matrix.indices.nbytes + real_tfidf_matrix.indptr.nbytes
print(f"TF-IDF Matrix: {tfidf_size / 1024 / 1024:.2f} MB")
print(f"  - Shape: {real_tfidf_matrix.shape}")
print(f"  - Sparsity: {(1 - real_tfidf_matrix.nnz / (real_tfidf_matrix.shape[0] * real_tfidf_matrix.shape[1])) * 100:.2f}%")

# Word2Vec storage
w2v_size = real_cbow.wv.vectors.nbytes
print(f"\nWord2Vec Embeddings: {w2v_size / 1024 / 1024:.2f} MB")
print(f"  - Vocabulary: {len(real_cbow.wv)}")
print(f"  - Dimensions: {real_cbow.vector_size}")

# FastText storage
ft_size = real_fasttext_cbow.wv.vectors.nbytes + real_fasttext_cbow.wv.vectors_ngrams.nbytes
print(f"\nFastText Embeddings: {ft_size / 1024 / 1024:.2f} MB")
print(f"  - Vocabulary: {len(real_fasttext_cbow.wv)}")
print(f"  - Dimensions: {real_fasttext_cbow.vector_size}")
print(f"  - N-grams: {len(real_fasttext_cbow.wv.vectors_ngrams)} additional vectors")

print("\n" + "="*60)
print("\n💡 Key Points:")
print("   - TF-IDF grows with number of documents (one vector per document)")
print("   - Embeddings grow with vocabulary size (one vector per word)")
print("   - For large corpora, embeddings are usually more efficient")
print("="*60)

---
# PART 3: Visualization and Comparison
---

In [ ]:
def visualize_embeddings(model, words, title):
    """Visualize word embeddings using PCA"""
    # Get vectors for words
    vectors = []
    valid_words = []
    
    for word in words:
        if hasattr(model, 'wv') and word in model.wv:
            vectors.append(model.wv[word])
            valid_words.append(word)
        elif hasattr(model, 'get_vector') and model.get_vector(word) is not None:
            vectors.append(model.get_vector(word))
            valid_words.append(word)
    
    if len(vectors) < 2:
        print(f"Not enough valid words for visualization in {title}")
        return
    
    # Reduce to 2D using PCA
    pca = PCA(n_components=2)
    vectors_2d = pca.fit_transform(vectors)
    
    # Plot
    plt.figure(figsize=(10, 6))
    plt.scatter(vectors_2d[:, 0], vectors_2d[:, 1], s=100, alpha=0.6)
    
    for i, word in enumerate(valid_words):
        plt.annotate(word, (vectors_2d[i, 0], vectors_2d[i, 1]), 
                    fontsize=12, alpha=0.8)
    
    plt.title(title)
    plt.xlabel('First Principal Component')
    plt.ylabel('Second Principal Component')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

In [ ]:
# Visualize dummy data models
dummy_words = ['cat', 'dog', 'pet', 'animal', 'mat', 'sat']
print("Visualizing embeddings from dummy data...")
visualize_embeddings(cbow_model, dummy_words, "CBOW Embeddings (Dummy Data)")
visualize_embeddings(skipgram_model, dummy_words, "Skip-gram Embeddings (Dummy Data)")
visualize_embeddings(fasttext_cbow, dummy_words, "FastText CBOW Embeddings (Dummy Data)")
visualize_embeddings(fasttext_skipgram, dummy_words, "FastText Skip-gram Embeddings (Dummy Data)")
visualize_embeddings(glove_model, dummy_words, "GloVe Embeddings (Dummy Data)")

In [ ]:
# Visualize real data models
if available_test_words:
    real_words = available_test_words[:10] if len(available_test_words) >= 10 else available_test_words
    print("Visualizing embeddings from real data...")
    visualize_embeddings(real_cbow, real_words, "CBOW Embeddings (AG News)")
    visualize_embeddings(real_skipgram, real_words, "Skip-gram Embeddings (AG News)")
    visualize_embeddings(real_fasttext_cbow, real_words, "FastText CBOW Embeddings (AG News)")
    visualize_embeddings(real_fasttext_skipgram, real_words, "FastText Skip-gram Embeddings (AG News)")

In [ ]:
# Visualize GloVe embeddings on real data
if available_glove_words:
    glove_viz_words = available_glove_words[:10] if len(available_glove_words) >= 10 else available_glove_words
    print("Visualizing GloVe embeddings on real data...")
    visualize_embeddings(real_glove, glove_viz_words, "GloVe Embeddings (AG News Subset)")

---
# PART 4: Model Comparison: Word2Vec vs FastText vs GloVe
---

## KEY DIFFERENCES (from lecture slides):

### CBOW (Continuous Bag of Words):
- Predicts target word from context words
- Faster to train
- Better for frequent words
- Smooths over distributional information
- Local context window approach

### Skip-gram:
- Predicts context words from target word
- Works better with rare words
- Better for small datasets
- Captures more detailed word relationships
- Local context window approach

### FastText:
- Extension of Word2Vec using character n-grams
- Can handle out-of-vocabulary (OOV) words
- Captures morphological information (prefixes, suffixes)
- Better for morphologically rich languages
- Slightly slower to train but more robust
- Works with both CBOW and Skip-gram architectures

### GloVe (Global Vectors):
- Uses global co-occurrence statistics
- Learns from word-word co-occurrence matrix
- Combines benefits of matrix factorization and local context
- Often produces high-quality embeddings
- Deterministic training (given same data)
- Can be slower to train on very large corpora

### Comparison Summary:

| Feature | Word2Vec | FastText | GloVe |
|---------|----------|----------|-------|
| OOV Handling | ❌ No | ✅ Yes | ❌ No |
| Training Speed | Fast | Medium | Slow |
| Morphology | ❌ No | ✅ Yes | ❌ No |
| Context Type | Local | Local | Global |
| Rare Words | Skip-gram better | Excellent | Good |
| Memory Usage | Low | Medium | High |

### In this notebook:
- Dummy data: All models work similarly on small vocabulary
- Real data: 
  - FastText excels at handling OOV and morphological variations
  - Skip-gram often captures more nuanced relationships
  - GloVe captures global corpus statistics
  - FastText provides most robust representations for varied vocabulary

### 4.1 Quantitative Comparison

In [ ]:
# Compare all models on common words
def compare_models_similarity(word, models_dict):
    """Compare similarity results across different models"""
    print(f"\n{'='*60}")
    print(f"Comparing models for word: '{word}'")
    print(f"{'='*60}\n")
    
    for model_name, model in models_dict.items():
        print(f"{model_name}:")
        try:
            if hasattr(model, 'wv'):
                if word in model.wv:
                    similar = model.wv.most_similar(word, topn=3)
                    for w, score in similar:
                        print(f"  {w}: {score:.4f}")
                else:
                    print(f"  Word '{word}' not in vocabulary")
            elif hasattr(model, 'most_similar'):
                similar = model.most_similar(word, topn=3)
                for w, score in similar:
                    print(f"  {w}: {score:.4f}")
        except Exception as e:
            print(f"  Error: {e}")
        print()

# Compare on dummy data
dummy_models = {
    "Word2Vec CBOW": cbow_model,
    "Word2Vec Skip-gram": skipgram_model,
    "FastText CBOW": fasttext_cbow,
    "FastText Skip-gram": fasttext_skipgram,
    "GloVe": glove_model
}

compare_models_similarity('cat', dummy_models)
compare_models_similarity('dog', dummy_models)

In [ ]:
# Compare on real data
if available_test_words:
    real_models = {
        "Word2Vec CBOW": real_cbow,
        "Word2Vec Skip-gram": real_skipgram,
        "FastText CBOW": real_fasttext_cbow,
        "FastText Skip-gram": real_fasttext_skipgram,
        "GloVe": real_glove
    }
    
    test_word = available_test_words[0]
    compare_models_similarity(test_word, real_models)

In [ ]:
# Test OOV handling comparison
print("\n" + "="*60)
print("OOV WORD HANDLING COMPARISON")
print("="*60 + "\n")

oov_word = 'businessman'  # Likely not in vocabulary
print(f"Testing OOV word: '{oov_word}'\n")

print("Word2Vec CBOW:")
try:
    if oov_word not in real_cbow.wv:
        _ = real_cbow.wv[oov_word]
except KeyError:
    print("  ❌ Cannot handle OOV word (KeyError)\n")

print("FastText CBOW:")
try:
    vec = real_fasttext_cbow.wv[oov_word]
    print("  ✅ Can generate vector using character n-grams")
    similar = real_fasttext_cbow.wv.most_similar(oov_word, topn=3)
    print("  Most similar:")
    for w, score in similar:
        print(f"    {w}: {score:.4f}")
except Exception as e:
    print(f"  Error: {e}")
print()

print("GloVe:")
if oov_word not in real_glove.word_to_id:
    print("  ❌ Cannot handle OOV word (not in vocabulary)")
else:
    print("  ✅ Word is in vocabulary")

---
# PART 5: Saving and Loading Models
---

In [ ]:
# Save models
print("Saving models...")
real_cbow.save("word2vec_cbow.model")
real_skipgram.save("word2vec_skipgram.model")
real_fasttext_cbow.save("fasttext_cbow.model")
real_fasttext_skipgram.save("fasttext_skipgram.model")

# Save GloVe embeddings (save as numpy arrays)
np.save("glove_embeddings.npy", real_glove.embeddings)

# Save vocabulary mappings
import pickle
with open("glove_vocab.pkl", "wb") as f:
    pickle.dump({
        'word_to_id': real_glove.word_to_id,
        'id_to_word': real_glove.id_to_word
    }, f)

print("All models saved successfully!")

In [ ]:
# Load models
print("Loading models...")
loaded_cbow = Word2Vec.load("word2vec_cbow.model")
loaded_fasttext = FastText.load("fasttext_cbow.model")

# Load GloVe
import pickle
loaded_glove = SimpleGloVe(vector_size=100)
loaded_glove.embeddings = np.load("glove_embeddings.npy")
with open("glove_vocab.pkl", "rb") as f:
    vocab_data = pickle.load(f)
    loaded_glove.word_to_id = vocab_data['word_to_id']
    loaded_glove.id_to_word = vocab_data['id_to_word']
    loaded_glove.vocab_size = len(loaded_glove.word_to_id)

print("All models loaded successfully!")

# Test loaded models
if available_test_words:
    test_word = available_test_words[0]
    print(f"\nTesting loaded models with '{test_word}':")
    
    print("\nWord2Vec CBOW:")
    for word, score in loaded_cbow.wv.most_similar(test_word, topn=3):
        print(f"  {word}: {score:.4f}")
    
    print("\nFastText CBOW:")
    for word, score in loaded_fasttext.wv.most_similar(test_word, topn=3):
        print(f"  {word}: {score:.4f}")
    
    if test_word in loaded_glove.word_to_id:
        print("\nGloVe:")
        for word, score in loaded_glove.most_similar(test_word, topn=3):
            print(f"  {word}: {score:.4f}")

### Summary: When to Use Each Model

**Use Word2Vec when:**
- You have clean, well-formed text
- Vocabulary is fixed and known
- Training speed is important
- Memory is limited

**Use FastText when:**
- Text contains typos or misspellings
- Working with morphologically rich languages
- Need to handle OOV words in production
- Working with user-generated content (social media, etc.)

**Use GloVe when:**
- You want to leverage global corpus statistics
- Training can be done offline once
- You have sufficient computational resources
- Deterministic results are important

**In Practice:**
- For most applications: Start with FastText (most robust)
- For research/analysis: Try all three and compare
- For production: Consider pre-trained embeddings first

---
# EXERCISES FOR STUDENTS
---

Try these exercises to deepen your understanding:

## Basic Exercises (Beginner)

### 1. Experiment with hyperparameters:
- Change `vector_size` (50, 100, 200, 300) - How does dimensionality affect quality?
- Change `window` size (2, 5, 10, 15) - How does context size impact learned relationships?
- For FastText: try different n-gram ranges:
  - `min_n=2, max_n=3` (short n-grams)
  - `min_n=3, max_n=6` (balanced)
  - `min_n=4, max_n=8` (long n-grams)
- For GloVe: experiment with `learning_rate` (0.01, 0.05, 0.1) and `epochs` (50, 100, 200)
- **Question:** Which settings give the best word similarities?

### 2. Explore word similarities:
- Pick different words and find their most similar neighbors
- Compare results across all models (Word2Vec, FastText, GloVe)
- Try words from different categories:
  - Common words: 'the', 'is', 'and'
  - Nouns: 'business', 'market', 'company'
  - Verbs: 'run', 'jump', 'think'
  - Adjectives: 'good', 'bad', 'beautiful'
- **Question:** Do different models capture different types of relationships?

### 3. Visualize semantic clusters:
- Create groups of related words:
  - Animals: ['cat', 'dog', 'bird', 'fish', 'lion', 'elephant']
  - Colors: ['red', 'blue', 'green', 'yellow', 'black', 'white']
  - Emotions: ['happy', 'sad', 'angry', 'excited', 'worried']
  - Numbers: ['one', 'two', 'three', 'four', 'five']
- Visualize each cluster using PCA or t-SNE
- **Question:** Do semantically related words cluster together?

## Intermediate Exercises

### 4. Test word analogies:
- Implement the classic analogy task: A is to B as C is to ?
- Examples to try:
  - king - man + woman ≈ queen
  - paris - france + germany ≈ berlin
  - big - bigger + small ≈ smaller
  - good - better + bad ≈ worse
- Code:
```python
result = model.wv.most_similar(positive=['king', 'woman'], negative=['man'], topn=1)
print(f"king - man + woman = {result[0][0]}")
```
- **Question:** Which model performs best on analogies? Why?

### 5. Out-of-Vocabulary (OOV) stress testing:
- Create challenging OOV words:
  - Misspellings: 'buisness', 'recieve', 'occured'
  - Compound words: 'mega-corporation', 'cyber-attack'
  - Informal text: 'looool', 'yasss', 'omgggg'
  - Domain-specific: 'COVID-19', 'blockchain', 'AI-powered'
- Test with Word2Vec (should fail) vs FastText (should work)
- **Question:** How well does FastText capture meaning of OOV words?

### 6. Compare CBOW vs Skip-gram:
- Train both architectures on the same data
- Compare their performance on:
  - Frequent words (e.g., 'the', 'is', 'business')
  - Rare words (words appearing 5-10 times)
  - Word analogies
- Measure training time for both
- **Question:** When does Skip-gram outperform CBOW?

### 7. Load different datasets:
Try training on different HuggingFace datasets:
```python
# Movie reviews
dataset = load_dataset('imdb', split='train[:5000]')

# Restaurant reviews
dataset = load_dataset('yelp_review_full', split='train[:5000]')

# Wikipedia articles (small version)
dataset = load_dataset('wikipedia', '20220301.simple', split='train[:1000]')

# Scientific papers
dataset = load_dataset('scientific_papers', 'arxiv', split='train[:1000]')
```
- **Question:** How does domain affect embedding quality?

## Advanced Exercises

### 8. Implement embedding evaluation:
Create a word similarity evaluation:
```python
# Define word pairs with human similarity scores (0-10)
word_pairs = [
    ('cat', 'dog', 8.0),
    ('cat', 'car', 1.0),
    ('king', 'queen', 9.0),
    ('man', 'woman', 8.0),
    ('computer', 'keyboard', 7.0),
]

# Compute correlation between model scores and human scores
from scipy.stats import spearmanr

def evaluate_model(model, pairs):
    human_scores = []
    model_scores = []
    for w1, w2, human_score in pairs:
        if w1 in model.wv and w2 in model.wv:
            model_score = model.wv.similarity(w1, w2)
            human_scores.append(human_score)
            model_scores.append(model_score * 10)  # Scale to 0-10
    
    correlation, pvalue = spearmanr(human_scores, model_scores)
    return correlation
```
- **Question:** Which model correlates best with human judgments?

### 9. Morphological analysis with FastText:
- Test FastText on morphologically related words:
  - Verbs: run → running → runner → runs → ran
  - Adjectives: happy → happier → happiest → happiness → unhappy
  - Nouns: nation → national → nationality → nationalize
- Compute similarity between base word and derivatives
- Visualize embeddings of morphological families
- **Question:** Does FastText capture morphological relationships?

### 10. Visualize with t-SNE:
Replace PCA with t-SNE for better 2D visualization:
```python
from sklearn.manifold import TSNE

def visualize_tsne(model, words, title):
    vectors = [model.wv[word] for word in words if word in model.wv]
    valid_words = [word for word in words if word in model.wv]
    
    tsne = TSNE(n_components=2, random_state=42, perplexity=min(30, len(vectors)-1))
    vectors_2d = tsne.fit_transform(vectors)
    
    plt.figure(figsize=(12, 8))
    plt.scatter(vectors_2d[:, 0], vectors_2d[:, 1], s=100, alpha=0.6)
    for i, word in enumerate(valid_words):
        plt.annotate(word, (vectors_2d[i, 0], vectors_2d[i, 1]), fontsize=12)
    plt.title(title)
    plt.show()
```
- **Question:** Does t-SNE reveal different patterns than PCA?

### 11. Compare with pre-trained embeddings:
Load and compare with pre-trained models:
```python
# Load pre-trained Word2Vec (Google News)
import gensim.downloader as api
pretrained_w2v = api.load('word2vec-google-news-300')

# Load pre-trained GloVe
pretrained_glove = api.load('glove-wiki-gigaword-100')

# Load pre-trained FastText
pretrained_ft = api.load('fasttext-wiki-news-subwords-300')
```
- Compare with your trained models
- **Question:** How much data is needed to match pre-trained quality?

### 12. Bias detection in embeddings:
Investigate social biases in word embeddings:
```python
# Gender bias
def compute_bias(model, word, male_word='man', female_word='woman'):
    male_sim = model.wv.similarity(word, male_word)
    female_sim = model.wv.similarity(word, female_word)
    return male_sim - female_sim

# Test occupations
occupations = ['doctor', 'nurse', 'engineer', 'teacher', 'programmer']
for occupation in occupations:
    if occupation in model.wv:
        bias = compute_bias(model, occupation)
        print(f"{occupation}: {bias:.4f} ({'male' if bias > 0 else 'female'} leaning)")
```
- **Question:** What biases exist in your embeddings?

### 13. Build a semantic search engine:
```python
def semantic_search(query, documents, model, top_k=5):
    # Compute document vectors (average word vectors)
    def doc_vector(text):
        words = text.lower().split()
        vectors = [model.wv[w] for w in words if w in model.wv]
        if vectors:
            return np.mean(vectors, axis=0)
        return None
    
    query_vec = doc_vector(query)
    if query_vec is None:
        return []
    
    # Compute similarities
    scores = []
    for doc in documents:
        doc_vec = doc_vector(doc)
        if doc_vec is not None:
            sim = cosine_similarity([query_vec], [doc_vec])[0][0]
            scores.append((doc, sim))
    
    # Return top-k
    return sorted(scores, key=lambda x: x[1], reverse=True)[:top_k]

# Test
query = "business news"
results = semantic_search(query, real_corpus[:100], real_cbow)
```
- **Question:** How does embedding quality affect search results?

## Challenge Exercises

### 14. Implement GloVe improvements:
- Add adaptive learning rate
- Implement mini-batch training
- Add different weighting functions
- Compare with baseline GloVe

### 15. Cross-lingual embeddings:
- Train embeddings on multilingual text
- Try to align embeddings from different languages
- Test translation capabilities

### 16. Dynamic embeddings:
- Implement a system that updates embeddings as new text arrives
- Compare static vs dynamic embedding quality
- Handle vocabulary growth

### 17. Contextualized embeddings comparison:
- Compare static embeddings (Word2Vec, FastText, GloVe) with contextualized embeddings
- Try simple context aggregation methods
- Analyze when context matters most

### 18. TF-IDF Experiments:
- Experiment with TF-IDF parameters:
  - `max_df` (0.5, 0.8, 0.95) - Remove common words
  - `min_df` (1, 5, 10) - Remove rare words
  - `max_features` (100, 1000, 5000) - Limit vocabulary
  - `ngram_range` - Try bigrams (1,2) or trigrams (1,3)
- **Question:** How do these parameters affect retrieval quality?

### 19. Hybrid TF-IDF + Embeddings:
Combine both approaches:
```python
def hybrid_similarity(query, documents, tfidf_model, embedding_model, alpha=0.5):
    # TF-IDF scores
    query_tfidf = tfidf_model.transform([query])
    tfidf_scores = cosine_similarity(query_tfidf, tfidf_matrix)[0]
    
    # Embedding scores
    query_emb = document_vector_from_embeddings(query, embedding_model)
    emb_scores = [...] # compute embedding similarities
    
    # Combine with weight alpha
    combined_scores = alpha * tfidf_scores + (1 - alpha) * emb_scores
    return combined_scores
```
- Test different alpha values (0.0 to 1.0)
- **Question:** Does combining approaches improve results?

### 20. TF-IDF for Feature Engineering:
- Use TF-IDF features in a classifier:
```python
from sklearn.linear_model import LogisticRegression

# Train classifier on TF-IDF features
clf = LogisticRegression()
clf.fit(tfidf_train, labels_train)
```
- Compare with embedding-based features
- **Question:** When does TF-IDF outperform embeddings for classification?

## Reflection Questions

After completing exercises, consider:

1. **Data size:** How much data is needed for quality embeddings?
2. **Architecture choice:** When should you choose CBOW vs Skip-gram vs FastText vs GloVe?
3. **Hyperparameters:** Which hyperparameters have the biggest impact?
4. **Evaluation:** How do you measure embedding quality in practice?
5. **Applications:** What downstream tasks benefit most from which embeddings?
6. **Limitations:** What are the fundamental limitations of static word embeddings?
7. **Ethics:** What biases exist in embeddings and how can they be mitigated?
8. **Future:** When should you use static embeddings vs transformer-based models?

---

## Bonus: Create Your Own Project

Choose one:
1. **Domain-specific embeddings:** Train embeddings on medical/legal/financial text
2. **Embedding-based classifier:** Build a text classifier using embeddings as features
3. **Analogy dataset:** Create and evaluate a custom analogy test set
4. **Visualization dashboard:** Build an interactive tool to explore embeddings
5. **Bias mitigation:** Implement and test a debiasing method
6. **Embedding compression:** Reduce embedding dimensions while preserving quality
7. **Multi-sense embeddings:** Handle words with multiple meanings
8. **Temporal embeddings:** Track how word meanings change over time

In [ ]:
# Your experiments here!
# Use this cell to implement your chosen exercises
# Good luck! 🚀
